<a href="https://colab.research.google.com/github/vaibhavmishra03/-datasciencecoursera/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load the Iris dataset
iris = load_iris()
X = iris.data


In [4]:
def evaluate_clustering(X, labels):
    if len(set(labels)) <= 1 or len(set(labels)) >= len(X):
        return [np.nan, np.nan, np.nan]
    return [
        round(silhouette_score(X, labels), 3),
        round(calinski_harabasz_score(X, labels), 0),
        round(davies_bouldin_score(X, labels), 2)
    ]


In [5]:
# Preprocessing configurations
preprocessing_methods = {
    'No Processing': lambda X: X,
    'Normalization': lambda X: MinMaxScaler().fit_transform(X),
    'Transform': lambda X: PowerTransformer().fit_transform(X),
    'PCA': lambda X: PCA(n_components=2).fit_transform(X),
    'T+N': lambda X: MinMaxScaler().fit_transform(PowerTransformer().fit_transform(X)),
    'T+N+PCA': lambda X: PCA(n_components=2).fit_transform(
        MinMaxScaler().fit_transform(PowerTransformer().fit_transform(X)))
}

cluster_range = [3, 4, 5]

In [6]:
# Initialize result dictionaries
results_kmeans = {}
results_hierarchical = {}
results_meanshift = {}

for method, preprocess in preprocessing_methods.items():
    X_proc = preprocess(X)
    for c in cluster_range:
        # K-Means
        kmeans = KMeans(n_clusters=c, random_state=42)
        kmeans_labels = kmeans.fit_predict(X_proc)
        results_kmeans[(method, c)] = evaluate_clustering(X_proc, kmeans_labels)

        # Hierarchical
        hier = AgglomerativeClustering(n_clusters=c)
        hier_labels = hier.fit_predict(X_proc)
        results_hierarchical[(method, c)] = evaluate_clustering(X_proc, hier_labels)

    # Mean Shift (uses bandwidth, doesn't need fixed 'c')
    bandwidth = estimate_bandwidth(X_proc, quantile=0.2)
    meanshift = MeanShift(bandwidth=bandwidth)
    ms_labels = meanshift.fit_predict(X_proc)
    results_meanshift[method] = evaluate_clustering(X_proc, ms_labels)

In [7]:
# Convert results to DataFrames
index = pd.MultiIndex.from_tuples(results_kmeans.keys(), names=['Preprocessing', 'Clusters'])
kmeans_df = pd.DataFrame(list(results_kmeans.values()), index=index, columns=['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin'])

index = pd.MultiIndex.from_tuples(results_hierarchical.keys(), names=['Preprocessing', 'Clusters'])
hierarchical_df = pd.DataFrame(list(results_hierarchical.values()), index=index, columns=['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin'])

meanshift_df = pd.DataFrame.from_dict(results_meanshift, orient='index', columns=['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin'])

# Display results
display(kmeans_df)
display(hierarchical_df)
display(meanshift_df)

# Optional: Save to CSV or plot heatmaps for visualization
kmeans_df.to_csv("kmeans_results.csv")
hierarchical_df.to_csv("hierarchical_results.csv")
meanshift_df.to_csv("meanshift_results.csv")

Silhouette  Calinski-Harabasz  Davies-Bouldin
Preprocessing Clusters                                               
No Processing 3              0.551              562.0            0.67
              4              0.498              530.0            0.75
              5              0.493              495.0            0.82
Normalization 3              0.483              351.0            0.79
              4              0.444              314.0            0.91
              5              0.423              263.0            0.99
Transform     3              0.490              162.0            0.82
              4              0.386              209.0            0.88
              5              0.369              170.0            0.89
PCA           3              0.598              694.0            0.56
              4              0.558              719.0            0.62
              5              0.510              642.0            0.75
T+N           3              0.481              358.0            0.80
              4              0.444              317.0            0.91
              5              0.417              265.0            0.98
T+N+PCA       3              0.536              464.0            0.67
              4              0.524              444.0            0.70
              5              0.527              389.0            0.68

Silhouette  Calinski-Harabasz  Davies-Bouldin
Preprocessing Clusters                                               
No Processing 3              0.554              558.0            0.66
              4              0.489              515.0            0.80
              5              0.484              488.0            0.82
Normalization 3              0.505              349.0            0.75
              4              0.433              301.0            0.85
              5              0.349              272.0            0.91
Transform     3              0.478              225.0            0.74
              4              0.427              214.0            0.90
              5              0.357              202.0            0.92
PCA           3              0.598              689.0            0.56
              4              0.541              674.0            0.65
              5              0.549              666.0            0.65
T+N           3              0.496              352.0            0.75
              4              0.431              304.0            0.86
              5              0.343              276.0            0.93
T+N+PCA       3              0.521              425.0            0.66
              4              0.502              417.0            0.76
              5              0.423              408.0            0.79

,Silhouette,Calinski-Harabasz,Davies-Bouldin
No Processing,0.686,510.0,0.39
Normalization,0.477,290.0,0.76
Transform,0.342,140.0,0.78
PCA,0.562,615.0,0.56
T+N,0.399,222.0,0.86
T+N+PCA,0.404,227.0,0.72
